In [8]:
#获取前一天的开盘价和收盘价
import struct
import os
from datetime import datetime, timedelta

def parse_tdx_day_file(file_path):
    """
    解析通达信日线数据文件(.day)
    文件格式：每32字节一条记录
    [0:4]日期, [4:8]开盘价, [8:12]最高价, [12:16]最低价, [16:20]收盘价, [20:24]成交额, [24:28]成交量
    """
    records = []
    try:
        with open(file_path, 'rb') as f:
            data = f.read()
            # 每条记录32字节
            record_size = 32
            num_records = len(data) // record_size
           # print(num_records)
            
            for i in range(num_records):
                start = i * record_size
                end = start + record_size
                record_data = data[start:end]
                
                # 解析二进制数据
                date_int, open_price, high, low, close, amount, volume, reserved = struct.unpack('<IIIIIIII', record_data)
             #   print(date_int, open_price, high, low, close, amount, volume)
                # 转换为正常价格（除以100）
                open_price = open_price / 100.0
                close = close / 100.0
                
                # 转换日期格式
                date_str = str(date_int)
                year = int(date_str[:4])
                month = int(date_str[4:6])
                day = int(date_str[6:8])
                record_date = datetime(year, month, day)
                
                records.append({
                    'date': record_date,
                    'open': open_price,
                    'high': high / 100.0,
                    'low': low / 100.0,
                    'close': close,
                    'amount': amount,
                    'volume': volume
                })
                
    except FileNotFoundError:
        print(f"文件未找到: {file_path}")
    except Exception as e:
        print(f"解析文件出错: {e}")
    
    return records

def get_previous_day_data(stock_code):
    """
    获取指定股票前一天的OHLC数据
    """
    # 根据股票代码确定市场前缀
    if stock_code.startswith('6'):
        market_prefix = 'sh'
    else:
        market_prefix = 'sz'
    
    # 通达信日线文件路径（需要根据实际安装路径调整）
    tdx_base_path = "D:/new_tdx/vipdoc"  # 通达信安装目录
    day_file_path = f"{tdx_base_path}/{market_prefix}/lday/{market_prefix}{stock_code}.day"
  #  print(day_file_path)
    
    # 解析日线文件
    records = parse_tdx_day_file(day_file_path)
    
    if not records:
        return None
    
    # 按日期排序，获取最近两天的数据
    records.sort(key=lambda x: x['date'], reverse=True)
    
    # 获取前一个交易日的数据（跳过当天如果是交易日的情况）
    today = datetime.now().date()
    previous_day_data = None
    
    for record in records:
        record_date = record['date'].date()
        if record_date < today:  # 找到最近的一个交易日
            previous_day_data = record
            break
    
    return previous_day_data

# 使用示例
if __name__ == "__main__":
    stock_code = "000001"  # 平安银行
    
    previous_data = get_previous_day_data(stock_code)
    if previous_data:
        print(f"股票 {stock_code} 前一日数据:")
        print(f"日期: {previous_data['date'].strftime('%Y-%m-%d')}")
        print(f"开盘价: {previous_data['open']:.2f}")
        print(f"收盘价: {previous_data['close']:.2f}")
        print(f"最高价: {previous_data['high']:.2f}")
        print(f"最低价: {previous_data['low']:.2f}")
    else:
        print("未找到前一日数据")

股票 000001 前一日数据:
日期: 2025-09-22
开盘价: 11.44
收盘价: 11.38
最高价: 11.47
最低价: 11.37


In [11]:
#获取当前盘中价格（实时数据）
import struct
import os
from datetime import datetime, timedelta

def parse_tdx_realtime_file(file_path, stock_code):
    """
    解析通达信分时数据文件
    注意：文件格式可能因通达信版本而异，以下为通用解析思路
    """
    try:
        if not os.path.exists(file_path) or os.path.getsize(file_path) == 0:
            print(f"文件不存在或为空: {file_path}")
            return None
        
        with open(file_path, 'rb') as f:
            data = f.read()
            
        # 分时数据通常包含多个时间点的数据记录
        # 每条记录的结构需要根据具体文件格式确定
        record_size = 32  # 常见记录大小，可能需要调整
        
        if len(data) < record_size:
            print("文件数据过短")
            return None
        
        # 取最后一条记录作为最新数据
        last_record_start = len(data) - record_size
        last_record = data[last_record_start:last_record_start + record_size]
        
        # 尝试解析价格信息（具体字节偏移需根据格式调整）
        # 以下为示例解析逻辑
        if len(last_record) >= 20:
            # 假设价格信息在特定偏移位置
            price_data = struct.unpack('<I', last_record[16:20])[0]  # 小端序无符号整型
            price = price_data / 100.0  # 除以100转换为实际价格
            
            return price
            
    except Exception as e:
        print(f"解析分时文件失败: {e}")
    
    return None

# 使用示例
def get_realtime_price(stock_code):
    """
    获取实时价格的整合函数
    """
    # 尝试多个可能的文件路径
    tdx_base_path = "D:/new_tdx"  # 请修改为您的通达信安装路径
    possible_paths = [
        f"{tdx_base_path}/T0002/hq_cache/sh.tfz",  # 上海市场缓存
        f"{tdx_base_path}/T0002/hq_cache/sz.tfz",  # 深圳市场缓存
      #  f"{tdx_base_path}/T0002/zst_cache/{'sh' if stock_code.startswith('6') else 'sz'}{stock_code}.zst",
    ]
    
    for path in possible_paths:
        price = parse_tdx_realtime_file(path, stock_code)
        if price:
            return price
    
    # 如果文件方式失败，尝试内存方式
    return 0

# 测试
if __name__ == "__main__":
    stock_code = "000001"  # 平安银行
    
    current_price = get_realtime_price(stock_code)
    print(current_price)
    

0


In [10]:
#检查数据
def get_stock_comprehensive_data(stock_code):
    """
    获取股票的完整数据：前一日OHLC + 当前实时价格
    """
    print(f"正在获取股票 {stock_code} 的数据...")
    
    # 获取前一日数据
    previous_data = get_previous_day_data(stock_code)
    
    # 获取当前实时价格
    current_price = get_realtime_price_from_tdx(stock_code)
    
    # 输出结果
    if previous_data:
        print("\n=== 前一日行情数据 ===")
        print(f"日期: {previous_data['date'].strftime('%Y-%m-%d')}")
        print(f"开盘价: {previous_data['open']:.2f}")
        print(f"收盘价: {previous_data['close']:.2f}")
        print(f"最高价: {previous_data['high']:.2f}")
        print(f"最低价: {previous_data['low']:.2f}")
        print(f"成交量: {previous_data['volume']:,} 手")
    else:
        print("前一日数据获取失败")
    
    if current_price:
        print(f"\n=== 当前实时价格 ===")
        print(f"当前价: {current_price:.2f}")
        
        if previous_data:
            change = current_price - previous_data['close']
            change_percent = (change / previous_data['close']) * 100
            print(f"涨跌: {change:+.2f} ({change_percent:+.2f}%)")
    else:
        print("实时价格获取失败")
    
    return {
        'previous_day': previous_data,
        'current_price': current_price
    }

# 主程序
if __name__ == "__main__":
    # 示例：获取平安银行的数据
    stock_data = get_stock_comprehensive_data("000001")
    
    # 可以基于这些数据进行交易决策
    if stock_data['previous_day'] and stock_data['current_price']:
        previous_close = stock_data['previous_day']['close']
        current_price = stock_data['current_price']
        
        # 简单的交易策略示例：当前价超过前收盘价1%时考虑买入
        if current_price > previous_close * 1.01:
            print("\n🔔 交易信号: 考虑买入")
        elif current_price < previous_close * 0.99:
            print("\n🔔 交易信号: 考虑卖出")

正在获取股票 000001 的数据...

=== 前一日行情数据 ===
日期: 2025-09-22
开盘价: 11.44
收盘价: 11.38
最高价: 11.47
最低价: 11.37
成交量: 59,640,232 手
实时价格获取失败
